In [17]:
from pyhive import hive
import warnings
warnings.filterwarnings('ignore')
conn = hive.Connection(host="jitsuOrchestrator", port=10000, auth="CUSTOM", username="computeai", password="infinite-scale")

In [18]:
import psutil

# Total and available memory in bytes
total_memory = psutil.virtual_memory().total
available_memory = psutil.virtual_memory().available

# Convert to gigabytes (optional)
total_memory_gb = total_memory / 1024**3
available_memory_gb = available_memory / 1024**3

print(f"VM: Total Memory: {total_memory_gb:.2f} GB")
print(f"VM: Available Memory: {available_memory_gb:.2f} GB")

# Number of logical and physical cores
logical_cores = psutil.cpu_count()
physical_cores = psutil.cpu_count(logical=False)

print(f"VM: Logical cores: {logical_cores}")
print(f"VM: Physical cores: {physical_cores}")

VM: Total Memory: 15.35 GB
VM: Available Memory: 9.62 GB
VM: Logical cores: 8
VM: Physical cores: 4


In [19]:
import boto3

def get_s3_object_count_and_size(bucket_name, prefix):
    s3 = boto3.client('s3')
    total_objects = 0
    total_size = 0

    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                total_objects += 1
                total_size += obj['Size']

    return total_objects, total_size

In [20]:
import pandas as pd

query = """
SELECT count(*) from parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/customer`
"""
df = pd.read_sql(query, conn)
print(df.to_markdown())

bucket_name = 'jitsu-test-data-east'
prefix = 'tpch/sf1000/parquet/partitioned/customer'

total_objects, total_size = get_s3_object_count_and_size(bucket_name, prefix)

print("Number of objects:", total_objects)
print("Total size:", total_size/1024/1024/1024, "GBs")  

|    |   count(1) |
|---:|-----------:|
|  0 |    1.5e+08 |
Number of objects: 187
Total size: 11.621752311475575 GBs


In [21]:
query = """
SELECT count(*) from parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/orders`
"""
df = pd.read_sql(query, conn)
print(df.to_markdown())

bucket_name = 'jitsu-test-data-east'
prefix = 'tpch/sf1000/parquet/partitioned/orders'

total_objects, total_size = get_s3_object_count_and_size(bucket_name, prefix)

print("Number of objects:", total_objects)
print("Total size:", total_size/1024/1024/1024, "GBs")  

|    |   count(1) |
|---:|-----------:|
|  0 |    1.5e+09 |
Number of objects: 1352
Total size: 62.80836641136557 GBs


In [22]:
query = """
SELECT count(*) from parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/lineitem`
"""
df = pd.read_sql(query, conn)
print(df.to_markdown())

bucket_name = 'jitsu-test-data-east'
prefix = 'tpch/sf1000/parquet/partitioned/lineitem'

total_objects, total_size = get_s3_object_count_and_size(bucket_name, prefix)

print("Number of objects:", total_objects)
print("Total size:", total_size/1024/1024/1024, "GBs")  

|    |    count(1) |
|---:|------------:|
|  0 | 5.99999e+09 |
Number of objects: 6069
Total size: 220.1947597367689 GBs


In [23]:
query = """
SELECT o.o_orderdate, SUM(l.l_quantity) as total_quantity, AVG(l.l_quantity) as avg_quantity, SUM(l.l_extendedprice) as total_price
FROM parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/customer` c
JOIN parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/orders` o ON c.c_custkey = o.o_custkey
JOIN parquet.`s3a://jitsu-test-data-east/tpch/sf1000/parquet/partitioned/lineitem` l ON o.o_orderkey = l.l_orderkey
GROUP BY o.o_orderdate
"""
df = pd.read_sql(query, conn)
print(df)

     o_orderdate  total_quantity  avg_quantity   total_price
0     1992-01-01      63584373.0     25.507578  9.535312e+10
1     1992-01-17      63661463.0     25.522757  9.545558e+10
2     1992-01-28      63544695.0     25.507757  9.528840e+10
3     1992-02-02      63626091.0     25.505416  9.543324e+10
4     1992-02-09      63484649.0     25.502039  9.517836e+10
...          ...             ...           ...           ...
2401  1998-04-18      63565013.0     25.492899  9.529262e+10
2402  1998-05-21      63651047.0     25.511176  9.544285e+10
2403  1998-06-06      63595162.0     25.496983  9.536310e+10
2404  1998-07-21      63593365.0     25.482307  9.533489e+10
2405  1998-07-26      63559827.0     25.496975  9.531659e+10

[2406 rows x 4 columns]


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assume we want to predict 'total_price' based on 'total_quantity' and 'avg_quantity'
X = df[['total_quantity', 'avg_quantity']]
y = df['total_price']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [25]:
# Use the model to make predictions on the test set
predictions = model.predict(X_test)
print(predictions)

[9.53284322e+10 9.53658446e+10 9.55019204e+10 9.54456758e+10
 9.53411438e+10 9.54375312e+10 9.55301532e+10 9.54518914e+10
 9.54096866e+10 9.52357687e+10 9.53195677e+10 9.52703917e+10
 9.53004828e+10 9.52630206e+10 9.53714716e+10 9.52609533e+10
 9.53815526e+10 9.54449703e+10 9.54620496e+10 9.53940676e+10
 9.54283011e+10 9.55455428e+10 9.53396985e+10 9.54151743e+10
 9.54318620e+10 9.54545085e+10 9.53128036e+10 9.52793414e+10
 9.53206686e+10 9.52411933e+10 9.54501986e+10 9.53879027e+10
 9.53714431e+10 9.51449414e+10 9.53857128e+10 9.53010158e+10
 9.53340077e+10 9.53157441e+10 9.52791522e+10 9.53341861e+10
 9.52253136e+10 9.54403976e+10 9.53543527e+10 9.53771469e+10
 9.52626316e+10 9.53586355e+10 9.53503019e+10 9.52674078e+10
 9.54421238e+10 9.53528963e+10 9.54527041e+10 9.52149897e+10
 9.56147689e+10 9.52403737e+10 9.54415486e+10 9.52808459e+10
 9.53111783e+10 9.55058309e+10 9.52979760e+10 9.53556581e+10
 9.54033179e+10 9.54501015e+10 9.54728974e+10 9.54589737e+10
 9.53298047e+10 9.544314